In [18]:
import pandas as pd 
import numpy as np
from dash_website.utils import BLUE_GREY_RED

CUSTOM_ORDER = [
    "*", 
    "*instances01", 
    "*instances1.5x", 
    "*instances23",
    "Brain",
    "Eyes",
    "Hearing",
    "Lungs",
    "Arterial",
    "Heart",
    "Abdomen",
    "Musculoskeletal",
    "PhysicalActivity",
    "Biochemistry",
    "BloodCells",
]

sample_definition = "all_samples_per_participant"
order_by = "custom"

data_custom_dimensions = pd.read_feather(f"../../all_data/correlation_between_accelerated_aging_dimensions/custom_dimensions_{sample_definition}.feather").to_dict()

In [46]:
def add_line_and_annotation(
    text, first_axis, second_axis, min_position, max_position, inner_margin, outer_margin, textangle, size, final=False
):
    if not final:
        to_match_heatmap = -10 / 2
        position = min_position
    else:
        to_match_heatmap = +10 / 2
        position = max_position
    return (
        {
            "type": "line",
            "xref": "x",
            "yref": "y",
            f"{first_axis}0": float(position + to_match_heatmap),
            f"{second_axis}0": inner_margin,
            f"{first_axis}1": float(position + to_match_heatmap),
            f"{second_axis}1": outer_margin,
            "line": {"color": "Black", "width": 0.5},
        },
        {
            "text": text,
            "xref": "x",
            "yref": "y",
            first_axis: float((min_position + max_position) / 2),
            second_axis: (inner_margin + outer_margin) / 2,
            "showarrow": False,
            "textangle": textangle,
            "font": {"size": size},
        },
    )

In [52]:
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap
import plotly.graph_objs as go

correlations = pd.DataFrame(data_custom_dimensions)

table_correlations = correlations.pivot(
    index=["dimension_1", "subdimension_1"],
    columns=["dimension_2", "subdimension_2"],
    values="correlation",
)

customdata_list = []
for customdata_item in ["correlation_std", "r2_1", "r2_std_1", "r2_2", "r2_std_2"]:
    customdata_list.append(
        correlations.pivot(
            index=["dimension_1", "subdimension_1"],
            columns=["dimension_2", "subdimension_2"],
            values=customdata_item,
        ).values
    )
stacked_customdata = list(map(list, np.dstack(customdata_list)))

customdata = pd.DataFrame(None, index=table_correlations.index, columns=table_correlations.columns)
customdata[customdata.columns] = stacked_customdata

hovertemplate = "Correlation: %{z:.3f} +- %{customdata[0]:.3f} <br><br>Dimensions 1: %{x} <br>r2: %{customdata[1]:.3f} +- %{customdata[2]:.3f} <br>Dimensions 2: %{y} <br>r2: %{customdata[3]:.3f} +- %{customdata[4]:.3f}<br><extra></extra>"

if order_by == "clustering" or 0 != 0:
    fig = create_dendrogram_heatmap(table_correlations, hovertemplate, customdata)
elif order_by == "r2" or 0 != 0:
    sorted_dimensions = (
        correlations.set_index(["dimension_1", "subdimension_1"])
        .sort_values(by="r2_1", ascending=False)
        .index.drop_duplicates()
    )

    sorted_table_correlations = table_correlations.loc[sorted_dimensions, sorted_dimensions]
    sorted_customdata = customdata.loc[sorted_dimensions, sorted_dimensions]

    heatmap = go.Heatmap(
        x=[" - ".join(elem) for elem in sorted_table_correlations.columns.values],
        y=[" - ".join(elem) for elem in sorted_table_correlations.index.values],
        z=sorted_table_correlations,
        colorscale=BLUE_GREY_RED,
        customdata=sorted_customdata,
        hovertemplate=hovertemplate,
        zmin=-1,
        zmax=1,
    )

    fig = go.Figure(heatmap)

else:  # order_by == "custom"
    sorted_dimensions = (
        correlations.set_index(["dimension_1", "subdimension_1"])
        .loc[CUSTOM_ORDER]
        .index.drop_duplicates()
    )
    
    sorted_table_correlations = table_correlations.loc[sorted_dimensions, sorted_dimensions]
    sorted_customdata = customdata.loc[sorted_dimensions, sorted_dimensions]

    heatmap = go.Heatmap(
        x=np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
        y=np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
        z=sorted_table_correlations,
        colorscale=BLUE_GREY_RED,
        customdata=sorted_customdata,
        hovertemplate=hovertemplate,
        zmin=-1,
        zmax=1,
    )

    fig = go.Figure(heatmap)

    fig.update_layout(
        xaxis={
            "tickvals": np.arange(5, 10 * sorted_table_correlations.shape[1] + 5, 10),
            "ticktext": [" - ".join(elem) for elem in sorted_table_correlations.columns.values],
        },
        yaxis={
            "tickvals": np.arange(5, 10 * sorted_table_correlations.shape[0] + 5, 10),
            "ticktext": [" - ".join(elem) for elem in sorted_table_correlations.index.values],
        },
    )

    dimensions = (
        sorted_table_correlations.index.to_frame()[["dimension_1", "subdimension_1"]]
        .reset_index(drop=True)
        .rename(
            columns={
                "dimension_1": "dimension",
                "subdimension_1": "subdimension"
            }
        )
    )
    dimensions["position"] = fig["layout"]["xaxis"]["tickvals"]
    dimensions.set_index(["dimension", "subdimension"], inplace=True)

    lines = []
    annotations = []

    for dimension in dimensions.index.get_level_values("dimension").drop_duplicates():
        dimension_inner_margin = -30
        dimension_outer_margin = -60

        min_position = dimensions.loc[dimension].min()
        max_position = dimensions.loc[dimension].max()

        for first_axis, second_axis in [("x", "y"), ("y", "x")]:
            if first_axis == "x":
                textangle = 90
            else:  # first_axis == "y"
                textangle = 0

            line, annotation = add_line_and_annotation(
                dimension,
                first_axis,
                second_axis,
                min_position,
                max_position,
                dimension_inner_margin,
                dimension_outer_margin,
                textangle,
                10,
            )

            lines.append(line)
            annotations.append(annotation)

            for subdimension in dimensions.loc[dimension].index.get_level_values("subdimension").drop_duplicates():
                subdimension_inner_margin = 0
                subdimension_outer_margin = -30

                submin_position = dimensions.loc[(dimension, subdimension)].min()
                submax_position = dimensions.loc[(dimension, subdimension)].max()

                for first_axis, second_axis in [("x", "y"), ("y", "x")]:
                    if first_axis == "x":
                        textangle = 90
                    else:  # first_axis == "y"
                        textangle = 0

                    line, annotation = add_line_and_annotation(
                        subdimension,
                        first_axis,
                        second_axis,
                        submin_position,
                        submax_position,
                        subdimension_inner_margin,
                        subdimension_outer_margin,
                        textangle,
                        8,
                    )

                    lines.append(line)
                    annotations.append(annotation)

    # The final top/right line
    for first_axis, second_axis in [("x", "y"), ("y", "x")]:
        line, _ = add_line_and_annotation(
            dimension,
            first_axis,
            second_axis,
            min_position,
            max_position,
            0,
            dimension_outer_margin,
            0,
            10,
            final=True,
        )

        lines.append(line)

    fig["layout"]["shapes"] = lines
    fig["layout"]["annotations"] = annotations

if order_by == "custom":
    fig.update_layout(yaxis={"showticklabels": False}, xaxis={"showticklabels": False})
else:
    fig.update_layout(font={"size": 8})

fig.update_layout(
    yaxis={"showgrid": False, "zeroline": False},
    xaxis={"showgrid": False, "zeroline": False},
    width=1100,
    height=1100,
)

fig

In [29]:
[sorted_dimensions.get_level_values("dimension_1"), sorted_dimensions.get_level_values("subdimension_1")]

[Index(['*', '*instances01', '*instances1.5x', '*instances23', 'Brain', 'Brain',
        'Brain', 'Eyes', 'Eyes', 'Eyes', 'Eyes', 'Hearing', 'Lungs', 'Arterial',
        'Arterial', 'Arterial', 'Heart', 'Heart', 'Heart', 'Abdomen', 'Abdomen',
        'Abdomen', 'Musculoskeletal', 'Musculoskeletal', 'Musculoskeletal',
        'Musculoskeletal', 'Musculoskeletal', 'Musculoskeletal',
        'PhysicalActivity', 'Biochemistry', 'Biochemistry', 'Biochemistry',
        'BloodCells'],
       dtype='object', name='dimension_1'),
 Index(['*', '*', '*', '*', '*', 'Cognitive', 'MRI', '*', 'All', 'Fundus',
        'OCT', '*', '*', '*', 'PulseWaveAnalysis', 'Carotids', '*', 'ECG',
        'MRI', '*', 'Liver', 'Pancreas', '*', 'Spine', 'Hips', 'Knees',
        'FullBody', 'Scalars', '*', '*', 'Urine', 'Blood', '*'],
       dtype='object', name='subdimension_1')]